In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup as bs

In [66]:
driver=webdriver.Chrome('./chromedriver.exe')
url='https://www.korea.kr/totalSearch.do?srchFlag=Y&keyword=%EB%B6%80%EB%8F%99%EC%82%B0&oldKeyword=%EB%B6%80%EB%8F%99%EC%82%B0&resultKeyword=%EB%B6%80%EB%8F%99%EC%82%B0&sort=date&scnt=1&punit=10&where=news_korea_new_v2&dateRange=all&range=ALL&op=and&multiFlag=Y&news_type=pol'
driver.get(url)
time.sleep(2)

검색어=input('정책브리핑 검색어 입력: ')
page=input('검색할 페이지수: ')

#새로운 검색어 입력
elem=driver.find_element(By.XPATH,'//*[@id="topSearchKeywordSr"]')
elem.clear()#기존 텍스트 삭제
elem.send_keys('검색어')
elem.send_keys(Keys.RETURN)
time.sleep(2)

#정책 뉴스로 이동
driver.find_element(By.XPATH,'//*[@id="news_korea_new_v2"]/div[1]/div/ul/li[2]/a').click()
time.sleep(2)

res=[bs(driver.page_source,'html.parser')]


#3페이지까지 페이지 소스 가져오기
for page in range(2,pages+1):
    if page%10==0:     #10의 배수일때 버튼을 클릭한다
        driver.find_element(By.LINK_TEXT,str(page)).click()
        time.sleep(2)
        tmp=driver.page_source
        res.append(bs(tmp))
        driver.find_element(By.XPATH,'//*[@id="paging"]/div/a[3]').click()
        
    else: #10의 배수가 아닌경우 그냥 수자를 찍는다
        driver.find_element(By.LINK_TEXT,str(page)).click()
        time.sleep(2)
        tmp=driver.page_source
        res.append(bs(tmp))
    
driver.close()
driver.quit()

C:\Users\yumei\AppData\Local\Temp\ipykernel_12256\433770357.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('./chromedriver.exe')


In [3]:
result=[]
for j in res:
    for i in j.select('span.text>strong>a'):
        result.append({
            '제목':i.text.replace('\t',''),
            '링크':'https://www.korea.kr'+i['href']
        })

df=pd.DataFrame(result)
display(df)

,제목,링크
0,\n환경·생활·청소년 분야 6개 과제 ‘도전.한국’ 아이디어 공모\n,https://www.korea.kr/news/policyNewsView.do?ne...
1,"\n코로나19 영문 예방접종 증명서, ‘정부24’에서 온라인 발급\n",https://www.korea.kr/news/policyNewsView.do?ne...
2,\n‘통합 데이터지도’ 확대·개편…29만 데이터 간편 확인\n,https://www.korea.kr/news/policyNewsView.do?ne...
3,"\n중기부, ‘산타마스크가 전하는 따뜻한 선물 상자’ 개봉\n",https://www.korea.kr/news/policyNewsView.do?ne...
4,\n‘사흘 연휴’ 소동이 남긴 것\n,https://www.korea.kr/news/cultureColumnView.do...
5,\n긴급재난지원금 현금화하면 환수…가맹점 차별대우도 단속\n,https://www.korea.kr/news/policyNewsView.do?ne...
6,\n‘밈’이 지배하는 시대를 관통하다\n,https://www.korea.kr/news/cultureColumnView.do...
7,\n드론봇·병영체험·위문열차 공연…지상군페스티벌 가볼까\n,https://www.korea.kr/news/policyNewsView.do?ne...
8,\n“올해 ‘한국관광의 별’ 추천해주세요”\n,https://www.korea.kr/news/policyNewsView.do?ne...
9,\n축구는 축구가 아니다\n,https://www.korea.kr/news/cultureColumnView.do...


In [156]:
res_a=[]
for i in df['링크']:
#     print(i)
    html=requests.get(i)
    soup=bs(html.text,'html.parser')
    title=soup.select_one('div.article-head>h1').text
    date=soup.select('div.info>span')[0].text
    writer=soup.select('div.info>span')[1].text.replace('\t','').replace('\r','')
    contents=soup.find('div','view-cont').text.replace('\n\n\n\n\n\n\n','')
    res_a.append([title,date,writer,contents])
#     break

df1=pd.DataFrame(res_a)
df1.columns=['제목','날짜','작성자','내용']
df1

,제목,날짜,작성자,내용
0,환경·생활·청소년 분야 6개 과제 ‘도전.한국’ 아이디어 공모,2022.08.09,행정안전부,\n행정안전부가 오는 10일부터 10월 28일까지 국민참여형 사회문제 해결사업인 ‘...
1,"코로나19 영문 예방접종 증명서, ‘정부24’에서 온라인 발급",2021.12.16,행정안전부,\n앞으로 코로나19 영문 예방접종 증명서를 정부24 누리집과 애플리케이션(앱)에서...
2,‘통합 데이터지도’ 확대·개편…29만 데이터 간편 확인,2021.02.02,과학기술정보통신부,\n약 29만 건의 데이터 댐에 모인 데이터를 쉽고 빠르게 찾고 활용할 수 있도록 ...
3,"중기부, ‘산타마스크가 전하는 따뜻한 선물 상자’ 개봉",2020.12.10,중소벤처기업부,\n‘산타마스크가 전하는 따뜻한 선물’을 슬로건으로 내건 ‘2020 크리스마스 마켓...
4,‘사흘 연휴’ 소동이 남긴 것,2020.07.31,한기봉 한국신문윤리위원회 윤리위원,“모라고 했는지 1도 몰으갰습니다” 군복 차림의 그가 이렇게 쓴 종이를 들었다. ...
5,긴급재난지원금 현금화하면 환수…가맹점 차별대우도 단속,2020.05.12,행정안전부,\n행정안전부는 긴급재난지원금 지급이 본격화됨에 따라 본래 정책 목적대로 시중에서 ...
6,‘밈’이 지배하는 시대를 관통하다,2019.11.01,김봉현 힙합 저널리스트/작가,밈(Meme). 인터넷 밈. 모두가 한 번 쯤은 들어본 단어다. \n밈의 기원을 ...
7,드론봇·병영체험·위문열차 공연…지상군페스티벌 가볼까,2018.10.04,육군,"\n대한민국 육군 최대의 축제, 제16회 지상군페스티벌이 5일부터 9일까지 5일간 ..."
8,“올해 ‘한국관광의 별’ 추천해주세요”,2018.08.22,문화체육관광부,\n지난해 12월 12일 오후 서울 중구 롯데호텔에서 열린 ‘2017 한국관광의 별...
9,축구는 축구가 아니다,2018.07.20,한기봉 언론중재위원/칼럼니스트,누구나 다 그들이야말로 진정한 챔피언이라고 말했다. 패자가 승자가 될 수 있는 경...


In [90]:
soup.find('article','article-wrap')

<article class="article-wrap">
<div class="article-head">
<input id="titleText" name="titleText" type="hidden" value="환경·생활·청소년 분야 6개 과제 ‘도전.한국’ 아이디어 공모"/>
<h1>환경·생활·청소년 분야 6개 과제 ‘도전.한국’ 아이디어 공모</h1>
<h2>국민참여형 사회문제 해결사업…포상금 지급·정책 반영</h2>
<div class="info">
<span>2022.08.09</span>
<span>행정안전부</span>
</div>
<div class="tool">
<a class="plus" href="#">글자크게</a>
<a class="minus" href="#">글자작게</a>
<a class="print" href="#" onclick="viewPrint(); return false;" target="_blank">인쇄</a>
<!-- 2019.10.10 시큐어 코딩 XSS -->
<a class="list" href="#goList" onclick="goList()">목록</a>
</div>
</div>
<div class="article-content">
<div class="view-cont">
<p>행정안전부가 오는 10일부터 10월 28일까지 국민참여형 사회문제 해결사업인 ‘도전.한국’ 국민 아이디어 공모를 시작한다고 9일 밝혔다.</p>
<p>이번 공모는 환경·생활·청소년 등 3개 분야 6개 과제를 대상으로 진행한다. 사회문제 해결에 관심 있는 국민·기업·단체 누구나 참여할 수 있다.</p>
<p>참여 희망자는 공모기간 내에 온(ON)국민소통(<a href="https://www.onsotong.go.kr" target="_blank">https://www.onsotong.go.kr</a>) 내 ‘도전.한국’ 게시판에서 참가신청서와 아이디어 기획서 등을 내려 받아 제출하면 된다.</p>
<p>‘도전.한국’은 2020년 처음 시작

In [113]:
df1['내용'][0]

'\n행정안전부가 오는 10일부터 10월 28일까지 국민참여형 사회문제 해결사업인 ‘도전.한국’ 국민 아이디어 공모를 시작한다고 9일 밝혔다.\n이번 공모는 환경·생활·청소년 등 3개 분야 6개 과제를 대상으로 진행한다.\xa0사회문제 해결에 관심 있는 국민·기업·단체 누구나 참여할 수 있다.\n참여 희망자는 공모기간 내에 온(ON)국민소통(https://www.onsotong.go.kr) 내 ‘도전.한국’ 게시판에서 참가신청서와 아이디어 기획서 등을 내려 받아 제출하면 된다.\n‘도전.한국’은 2020년 처음 시작해 그 동안 총 85건의 우수 아이디어를 정부정책에 반영했다. 주요 정책으로는 ‘생체인식 기술을 활용한 반려동물 등록’, ‘전동킥보드 거치대 설치’ 등이다.\n\n이번 공모과제는 민간플랫폼의 ‘핵심검색어 데이터 분석’과 최근 실시한 ‘국민선호도 조사’ 결과 등을 반영해 국민이 시급하게 해결해야 한다고 생각하는 국가적 과제로 골랐다.\n총 3276명이 참여한 국민선호도 조사 결과 일회용품 사용 감축·재활용 방안(환경), 층간소음 문제 해결 방안(생활), 소년범죄 예방 및 사회복귀 지원 방안(청소년)이 각 분야별 최다 표를 얻어 공모과제로 선정됐다.\n접수된 아이디어는 오는 11월 ‘도전.한국’ 운영위원회의 심사를 거쳐 과제별 우수 아이디어를 선정하고, 12월 중 시상 및 포상금을 지급한다.\n포상금은 과제 난이도 및 아이디어 수준에 따라 결정하며, 선정한 아이디어 중 가장 우수하고 획기적인 아이디어 1건은 운영위원회 의결로 대상을 수여한다.\n특히 선정한 우수 아이디어는 전문기관을 통해 아이디어 숙성·고도화를 진행한 뒤 각 부처 정책에 반영하거나 특허등록 등 사업화를 지원한다.\n\n이동옥 행안부 정부혁신기획관은 “국민이 제안한 창의적 아이디어는 우리 사회가 풀기 어려운 난제를 해결하는데 큰 도움이 될 것”이라며 “온 국민의 집단지성을 모아 사회적 난제를 해결할 수 있도록 많은 관심과 참여를 기대한다”고 밝혔다.\n[붙임] 2022년 ‘도전.한국’ 공

In [132]:
res_list=[]
for i in df['링크']:
    print(i)

https://www.korea.kr/news/policyNewsView.do?newsId=148904538
https://www.korea.kr/news/policyNewsView.do?newsId=148896914
https://www.korea.kr/news/policyNewsView.do?newsId=148883476
https://www.korea.kr/news/policyNewsView.do?newsId=148880827
https://www.korea.kr/news/cultureColumnView.do?newsId=148875331
https://www.korea.kr/news/policyNewsView.do?newsId=148872361
https://www.korea.kr/news/cultureColumnView.do?newsId=148866194
https://www.korea.kr/news/policyNewsView.do?newsId=148854324
https://www.korea.kr/news/policyNewsView.do?newsId=148853333
https://www.korea.kr/news/cultureColumnView.do?newsId=148852604
https://www.korea.kr/news/healthView.do?newsId=148852045
https://www.korea.kr/news/policyNewsView.do?newsId=148849960
https://www.korea.kr/news/cultureColumnView.do?newsId=148849428
https://www.korea.kr/news/policyNewsView.do?newsId=148846215
https://www.korea.kr/news/policyNewsView.do?newsId=148845048
https://www.korea.kr/news/policyNewsView.do?newsId=148829367
https://www.kore